In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf

from data_augmentation import TEST_DATA, VALID_DATA, BATCH_SIZE, MAX_LEN, data_generator
from templates import allowed_chars

2024-02-25 13:48:19.776978: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-25 13:48:20.039416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 13:48:20.039511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 13:48:20.077803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 13:48:20.164711: I tensorflow/core/platform/cpu_feature_guar

In [2]:
output_signature = (
    (tf.TensorSpec(shape=(BATCH_SIZE, MAX_LEN), dtype=tf.float32),
     tf.TensorSpec(shape=(BATCH_SIZE, MAX_LEN), dtype=tf.float32)),
    tf.TensorSpec(shape=(BATCH_SIZE,), dtype=tf.int32)
)

address_dataset = tf.data.Dataset.from_generator(data_generator,
                                                 output_signature=output_signature)

2024-02-25 13:51:36.381892: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-25 13:51:36.520507: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-25 13:51:36.520820: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
vocab_size = len(allowed_chars) + 1  # +1 for padding token
embedding_dim = 5

embedding_layer = Embedding(vocab_size, embedding_dim, input_length=MAX_LEN)
bilstm_layer = Bidirectional(LSTM(128, dropout=0.05, recurrent_dropout=0.05))

# Define two input layers
input_a = Input(shape=(MAX_LEN,))
input_b = Input(shape=(MAX_LEN,))

# Shared embedding layer
embedded_a = embedding_layer(input_a)
embedded_b = embedding_layer(input_b)

# Shared BiLSTM layer
processed_a = bilstm_layer(embedded_a)
processed_b = bilstm_layer(embedded_b)

# Concatenation of the two outputs
concatenated = Concatenate()([processed_a, processed_b])

# Fully connected layers
fc_layer_1 = Dense(64, activation='relu')(concatenated)
fc_layer_1 = Dropout(0.5)(fc_layer_1)
fc_layer_2 = Dense(32, activation='relu')(fc_layer_1)
fc_layer_2 = Dropout(0.1)(fc_layer_2)
output_layer = Dense(1, activation='sigmoid')(fc_layer_2)

# Building the model
model = Model(inputs=[input_a, input_b], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 120)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 120)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 120, 5)               250       ['input_3[0][0]',             
                                                                     'input_4[0][0]']             
                                                                                                  
 bidirectional_1 (Bidirecti  (None, 256)                  137216    ['embedding_1[0][0]',   

In [6]:
steps_per_epoch = 2**11
epochs = 50

checkpoint_filepath = 'model_checkpoints/my_model_weights_best.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True)


model.fit(address_dataset, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=VALID_DATA, callbacks=[model_checkpoint_callback])

Epoch 1/50
2047/2048 [============================>.] - ETA: 0s - loss: 0.4600 - accuracy: 0.7669
Epoch 1: val_accuracy improved from -inf to 0.93903, saving model to model_checkpoints/my_model_weights_best.keras
2048/2048 [==============================] - 38s 17ms/step - loss: 0.4599 - accuracy: 0.7670 - val_loss: 0.1850 - val_accuracy: 0.9390
Epoch 2/50
2048/2048 [==============================] - ETA: 0s - loss: 0.1355 - accuracy: 0.9539
Epoch 2: val_accuracy improved from 0.93903 to 0.97241, saving model to model_checkpoints/my_model_weights_best.keras
2048/2048 [==============================] - 35s 17ms/step - loss: 0.1355 - accuracy: 0.9539 - val_loss: 0.0770 - val_accuracy: 0.9724
Epoch 3/50
2048/2048 [==============================] - ETA: 0s - loss: 0.0905 - accuracy: 0.9690
Epoch 3: val_accuracy improved from 0.97241 to 0.97925, saving model to model_checkpoints/my_model_weights_best.keras
2048/2048 [==============================] - 35s 17ms/step - loss: 0.0905 - accuracy:

In [7]:
best_model = tf.keras.models.load_model('model_checkpoints/my_model_weights_best.keras')

In [10]:
test_predictions = best_model.predict(TEST_DATA[0])

10/10 [==============================] - 1s 12ms/step


array([[1.16509181e-02],
       [1.64270954e-04],
       [6.77690914e-05],
       [8.90602647e-10],
       [6.86086699e-09],
       [3.45905721e-10],
       [1.38221794e-05],
       [1.26401492e-05],
       [2.07014081e-20],
       [1.56696586e-10],
       [3.91011196e-10],
       [1.39061397e-13],
       [3.63835125e-15],
       [5.87576591e-14],
       [1.53945277e-11],
       [1.81796779e-07],
       [1.77151072e-04],
       [1.43284039e-12],
       [2.31816056e-16],
       [1.67447354e-08],
       [1.63651759e-08],
       [5.76673001e-16],
       [1.12914896e-14],
       [3.51612175e-06],
       [2.82999419e-04],
       [5.14564248e-13],
       [1.65580101e-13],
       [2.13576872e-02],
       [4.89100785e-07],
       [5.59272806e-09],
       [1.68796064e-12],
       [1.13358418e-07],
       [2.16972614e-15],
       [6.22412699e-09],
       [5.80649721e-12],
       [7.03506808e-10],
       [2.51899755e-14],
       [2.66089641e-14],
       [6.37806821e-11],
       [2.83349073e-06],


In [12]:
best_model.evaluate(*TEST_DATA)

10/10 [==============================] - 1s 10ms/step - loss: 0.7067 - accuracy: 0.9281


[0.7067360281944275, 0.9281250238418579]